In [4]:
import os,sys,glob
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
# os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import numpy as np
import pandas as pd
import shutil
import pickle
import tqdm
from tqdm import tqdm
from random import shuffle
from math import log, floor
from keras.utils import *
from keras.utils.generic_utils import *
from keras.preprocessing.text import *
from keras.preprocessing.sequence import *
from keras.preprocessing.image import *
from multiprocessing import *
import gensim
from gensim.models.word2vec import *
from sklearn.metrics.pairwise import *
from sklearn.model_selection import *
from sklearn.manifold import *
from sklearn.decomposition import *
from sklearn.cluster import *
from sklearn import preprocessing
import sent2vec
import re
import string
import unicodedata as udata
import matplotlib
import matplotlib.pyplot as plt
from collections import Counter
from operator import itemgetter
from collections import OrderedDict
import random

Using TensorFlow backend.


In [5]:
model = sent2vec.Sent2vecModel()
model.load_model('model/o2o_o2m_Sent2Vec_lower_woParam_0616_768.bin') # sent2vec model
dil= r"[,.;\-+^()/@#?!&$:{}\\*%~\'\"\=\_]+\ *" #等號、底線被保留，注意要跟先前一致
# all_df = pd.read_csv('./data/tree-rep-profiles-partial/process2family_df.csv')
api_li = ['LoadLibrary',
'CreateProcess',
'OpenProcess',
'ExitProcess',
'TerminateProcess',
'WinExec',
'CreateRemoteThread',
'CreateThread',
'CopyFile',
'CreateFile',
'DeleteFile',
'RegSetValue',
'RegCreateKey',
'RegDeleteKey',
'RegDeleteValue',
'RegQueryValue',
'RegEnumValue',
'WinHttpConnect',
'WinHttpOpen',
'WinHttpOpenRequest',
'WinHttpReadData',
'WinHttpSendRequest',
# 'WinHttpWriteData', #少了
'InternetOpen',
'InternetConnect',
'HttpSendRequest',
'GetUrlCacheEntryInfo']
api_li = [x.lower() for x in api_li] #lowrer case?
len(api_li)

26

In [6]:
encode_dict = {}
for i,v in enumerate(api_li):
    encode_dict[v] = i+1
pickle.dump(file=open('data/tree-rep-profiles_o2o/encode_dict.pkl','wb'),obj=encode_dict)
encode_dict
# encode_dict = pickle.load(open('data/tree-rep-profiles-partial/encode_dict.pkl','rb'))
# encode_dict

{'loadlibrary': 1,
 'createprocess': 2,
 'openprocess': 3,
 'exitprocess': 4,
 'terminateprocess': 5,
 'winexec': 6,
 'createremotethread': 7,
 'createthread': 8,
 'copyfile': 9,
 'createfile': 10,
 'deletefile': 11,
 'regsetvalue': 12,
 'regcreatekey': 13,
 'regdeletekey': 14,
 'regdeletevalue': 15,
 'regqueryvalue': 16,
 'regenumvalue': 17,
 'winhttpconnect': 18,
 'winhttpopen': 19,
 'winhttpopenrequest': 20,
 'winhttpreaddata': 21,
 'winhttpsendrequest': 22,
 'internetopen': 23,
 'internetconnect': 24,
 'httpsendrequest': 25,
 'geturlcacheentryinfo': 26}

## Build Data

In [28]:
def Sent2vec_emb(root_dir,dev=False,max_length=216,model=model,dim=768,encode_dict=encode_dict,dil=dil):
    # root_dir = './data/tree-rep-profiles_one2many/'
#     samples_pid_aliases = []
#     processed_pid = []
#     rasmma_dir = next(os.walk(root_dir))[1]
#     samples_pid_valid = valid_pids_df.index.tolist()
#     samples_pid_test = test_pids_df.index.tolist()
    # max_length = 347
    # dim = 768

    dev_pids_path = [] #儲存profile順序?
    
    sent_pad = [0]*dim
    # encode_dict = encode_dict_lower
    all_profiles = []
    all_api_name = []
    all_fam_ans = []
    all_byterep=[]
    hl_rep = []
    sen2vec_length_normalize = []
    sen2vec_dim_normalize = []
    loner = False
    if root_dir.split('/')[-2] in 'EXP': #partial match EXP dir
        loner = True
#     for rasmma in rasmma_dir:
    fam_dir = next(os.walk(root_dir))[1]
    for fam in tqdm(fam_dir):
        tree_dir = next(os.walk(root_dir +fam))[1]
        for tree in tree_dir:
            in_directory = root_dir + fam +  '/' + tree + '/'
            hl_list = next(os.walk(in_directory))[2]
            hl_list = [os.path.join(in_directory, f) for f in hl_list]
            hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
            for hl_f in hl_list:
                profile_emb = [] #整個api invocation call
                func_name_emb = [] #僅有api function name
                if not loner:
                    byterep = hl_f.replace('.profile','_byterep.pickle') #拿出hl_f所對應的byterep
                    rep = pickle.load(open(byterep,'rb')) #list type
                    if len(rep) < max_length: #Y1
                        for _ in range(max_length-len(rep)):
                            rep.append(0)
                with open(hl_f,encoding='ISO 8859-1') as f: #X2
                    lines = f.read()
                lines = re.sub(r'[^\x00-\x7F]+','', lines)
                lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
                lines = lines.splitlines()
                if len(lines) > max_length:
                    print("ERR length too long:",hl_f,'=>',len(lines))
                    continue
                dev_pids_path.append(hl_f)
                for line in lines:
                    temp = re.sub(dil," ",line.lower()) # lower? 跟先前一致
                    temp = temp.split(" ")
                    temp = list(filter(None, temp))
                    temp = ' '.join(temp)
                    func_name = temp.split(' ')[0] # X1
                    if func_name not in api_li:
                        print('=ERROR:=',hl_f,'=>',temp)
                    func_name_emb.append(encode_dict[func_name]) # encode証術數數字
                    emb = model.embed_sentence(temp)
                    emb = emb[0]
                    profile_emb.append(emb) #放入embdding，句子=>profile
                    sen2vec_dim_normalize.append(emb) #dim norm用
                sen2vec_length_normalize.append(len(lines))
                if len(lines) < max_length:
                    for _ in range(max_length-len(lines)):
                        profile_emb.append(sent_pad)
                if not loner:
                    hl_rep.append(rep) #Y1
                all_profiles.append(profile_emb) #X2
                all_api_name.append(func_name_emb) #X1
    #             all_byterep.extend(rep_list)
    all_api_name = pad_sequences(all_api_name,maxlen=max_length,padding='post',value=0) #X1

    if not loner:
        return np.array(all_api_name) , np.array(all_profiles) , np.array(hl_rep),np.array(sen2vec_dim_normalize) ,sen2vec_length_normalize, dev_pids_path
    else:
        return np.array(all_api_name) , np.array(all_profiles) ,np.array(sen2vec_dim_normalize) ,sen2vec_length_normalize,dev_pids_path


In [8]:
train_dir = './data/tree-rep-profiles_o2o/normal/'
train_api,train_emb, train_rep, train_normalize , train_len_norm,train_pids_path =   Sent2vec_emb(train_dir)
print(train_api.shape,train_emb.shape,train_rep.shape)
valid_dir = './data/tree-rep-profiles_o2o/DEV/'
valid_api,valid_emb, valid_rep, valid_normalize , valid_len_norm,valid_pids_path =   Sent2vec_emb(valid_dir)
print(valid_api.shape,valid_emb.shape,valid_rep.shape)
test_dir = './data/tree-rep-profiles_o2o/TEST/'
test_api,test_emb, test_rep, test_normalize , test_len_norm,test_pids_path =   Sent2vec_emb(test_dir)
print(test_api.shape,test_emb.shape,test_rep.shape)



  2%|▏         | 1/45 [00:00<00:05,  8.58it/s]

(7103, 216) (7103, 216, 768) (7103, 216)


  2%|▏         | 1/45 [00:00<00:07,  6.28it/s]

(451, 216) (451, 216, 768) (451, 216)


100%|██████████| 32/32 [00:00<00:00, 27060.03it/s]

(506, 216) (506, 216, 768) (506, 216)
(0, 216) (0,)


In [29]:
exp_dir = './data/tree-rep-profiles_o2o/EXP/'
exp_api,exp_emb, exp_normalize , exp_len_norm,exp_pids_path =   Sent2vec_emb(exp_dir)
print(exp_api.shape,exp_emb.shape)

100%|██████████| 32/32 [00:05<00:00,  3.62it/s]


(318, 216) (318, 216, 768)


## dim normalize

In [30]:
alls = np.concatenate([train_normalize,valid_normalize,test_normalize,exp_normalize])
mean = np.mean(alls,axis=0)
std = np.std(alls,axis=0)
mean.shape , std.shape

((768,), (768,))

In [31]:
def normalize_dim(emb,length,mean=mean,std=std):
    sent_emb = (emb[:length,:] - mean)/std
    final_emb = np.concatenate([sent_emb, emb[length:,:]],axis=0)
    return final_emb

In [32]:
test_emb_norm = []
valid_emb_norm = []
train_emb_norm = []
exp_emb_norm = []
for emb,length in zip(train_emb,train_len_norm):
    emb_norm = normalize_dim(emb,length)
    train_emb_norm.append(emb_norm)
for emb,length in zip(valid_emb,valid_len_norm):
    emb_norm = normalize_dim(emb,length)
    valid_emb_norm.append(emb_norm)
for emb,length in zip(test_emb,test_len_norm):
    emb_norm = normalize_dim(emb,length)
    test_emb_norm.append(emb_norm)
for emb,length in zip(exp_emb,exp_len_norm):
    emb_norm = normalize_dim(emb,length)
    exp_emb_norm.append(emb_norm)
train_emb = np.array(train_emb_norm)
valid_emb = np.array(valid_emb_norm)
test_emb = np.array(test_emb_norm)
exp_emb = np.array(exp_emb_norm)
print(train_emb.shape,valid_emb.shape,test_emb.shape,exp_emb.shape)

(7103, 216, 768) (451, 216, 768) (506, 216, 768) (318, 216, 768)


In [33]:
root_dir = './data/tree-rep-profiles_o2o/'
pickle.dump(file=open(root_dir + 'pids_train.pkl','wb')
            ,obj=(train_api , train_emb , train_rep),protocol=4)
pickle.dump(file=open(root_dir + 'pids_valid.pkl','wb')
            ,obj=(valid_api,valid_emb,valid_rep),protocol=4)
pickle.dump(file=open(root_dir + 'pids_test.pkl','wb')
            ,obj=(test_api,test_emb,test_rep),protocol=4)
pickle.dump(file=open(root_dir + 'pids_exp.pkl','wb')
            ,obj=(exp_api,exp_emb),protocol=4)